Entradas: Month, GHI, Wind Speed, Zenith, Azimuth, Hour Sine, Hour Cosine 

Target: Angulo

Dados do ano de 2019 inteiro


In [1]:
import pandas as pd
import pvlib
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv') 
# otimo e theta calculados com pvlib
angulos = pd.read_csv('otimo.csv')
thetas = pd.read_csv('backtracking.csv')
# zenite e azimute do get solar position
solpos = pd.read_csv('solarposition.csv')

# Filtrar o df e pegar dados de 04/07 até 18/11 (5760 dados)
#df_mes = df[(df['datetime'] >= '2019-07-22 00:00:00') & (df['datetime'] <= '2019-11-18 23:30:00')].copy()
df_mes = df.copy()
display(df_mes)
angulos_mes = angulos.copy()
display(angulos_mes)
df_mes['Angulo'] = angulos_mes['angulo_otimo']
thetas_mes = thetas.copy()
display(thetas_mes)
solpos_mes = solpos.copy()
display(solpos_mes)

,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,Temperature,DNI,GHI,Solar Zenith Angle,Wind Speed,Surface Albedo,Cloud Type,Fill Flag,datetime,datetime_utc,datetime_local
0,0,2019,1,1,0,0,0,20.9,0,0,133.35,2.4,0.16,0,0,2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2019-01-01 00:00:00-02:00
1,1,2019,1,1,0,30,0,20.7,0,0,133.16,2.3,0.16,0,0,2019-01-01 00:30:00,2019-01-01 00:30:00+00:00,2019-01-01 00:30:00-02:00
2,2,2019,1,1,1,0,0,20.6,0,0,131.85,2.2,0.16,0,0,2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2019-01-01 01:00:00-02:00
3,3,2019,1,1,1,30,0,20.5,0,0,129.51,2.0,0.16,0,0,2019-01-01 01:30:00,2019-01-01 01:30:00+00:00,2019-01-01 01:30:00-02:00
4,4,2019,1,1,2,0,0,20.4,0,0,126.28,1.9,0.16,0,0,2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2019-01-01 02:00:00-02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,2019,12,31,21,30,0,21.4,0,0,119.37,2.8,0.16,0,0,2019-12-31 21:30:00,2019-12-31 21:30:00+00:00,2019-12-31 21:30:00-03:00
17516,17516,2019,12,31,22,0,0,21.3,0,0,123.75,2.8,0.16,0,0,2019-12-31 22:00:00,2019-12-31 22:00:00+00:00,2019-12-31 22:00:00-03:00
17517,17517,2019,12,31,22,30,0,21.2,0,0,127.48,2.7,0.16,0,0,2019-12-31 22:30:00,2019-12-31 22:30:00+00:00,2019-12-31 22:30:00-03:00
17518,17518,2019,12,31,23,0,0,21.1,0,0,130.42,2.7,0.16,0,0,2019-12-31 23:00:00,2019-12-31 23:00:00+00:00,2019-12-31 23:00:00-03:00


,Unnamed: 0,datetime,angulo_otimo,irradiancia
0,0,2019-01-01 00:00:00,15,0.0
1,1,2019-01-01 00:30:00,15,0.0
2,2,2019-01-01 01:00:00,15,0.0
3,3,2019-01-01 01:30:00,15,0.0
4,4,2019-01-01 02:00:00,15,0.0
...,...,...,...,...
17515,17515,2019-12-31 21:30:00,60,0.0
17516,17516,2019-12-31 22:00:00,15,0.0
17517,17517,2019-12-31 22:30:00,15,0.0
17518,17518,2019-12-31 23:00:00,15,0.0


,datetime_local,tracker_theta
0,2019-01-01 00:00:00-02:00,0.0
1,2019-01-01 00:30:00-02:00,0.0
2,2019-01-01 01:00:00-02:00,0.0
3,2019-01-01 01:30:00-02:00,0.0
4,2019-01-01 02:00:00-02:00,0.0
...,...,...
17515,2019-12-31 21:30:00-03:00,0.0
17516,2019-12-31 22:00:00-03:00,0.0
17517,2019-12-31 22:30:00-03:00,0.0
17518,2019-12-31 23:00:00-03:00,0.0


,datetime_local,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
0,2019-01-01 00:00:00-02:00,130.431269,130.431269,-40.431269,-40.431269,201.247715,-3.235491
1,2019-01-01 00:30:00-02:00,132.428602,132.428602,-42.428602,-42.428602,192.435013,-3.245391
2,2019-01-01 01:00:00-02:00,133.361499,133.361499,-43.361499,-43.361499,183.099195,-3.255288
3,2019-01-01 01:30:00-02:00,133.166114,133.166114,-43.166114,-43.166114,173.623787,-3.265183
4,2019-01-01 02:00:00-02:00,131.856204,131.856204,-41.856204,-41.856204,164.429482,-3.275076
...,...,...,...,...,...,...,...
17515,2019-12-31 21:30:00-03:00,119.377718,119.377718,-29.377718,-29.377718,222.640064,-3.091657
17516,2019-12-31 22:00:00-03:00,123.755981,123.755981,-33.755981,-33.755981,216.380291,-3.101587
17517,2019-12-31 22:30:00-03:00,127.486511,127.486511,-37.486511,-37.486511,209.238066,-3.111515
17518,2019-12-31 23:00:00-03:00,130.422052,130.422052,-40.422052,-40.422052,201.206076,-3.121441


In [3]:
# Adicionando colunas de outros dfs ao df
colunas_desejadas = ['zenith', 'azimuth']
coluna = ['tracker_theta']
df_mes[colunas_desejadas] = solpos_mes[colunas_desejadas]
df_mes[coluna] = thetas_mes[coluna]
display(df_mes)


,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,Temperature,DNI,GHI,...,Surface Albedo,Cloud Type,Fill Flag,datetime,datetime_utc,datetime_local,Angulo,zenith,azimuth,tracker_theta
0,0,2019,1,1,0,0,0,20.9,0,0,...,0.16,0,0,2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2019-01-01 00:00:00-02:00,15,130.431269,201.247715,0.0
1,1,2019,1,1,0,30,0,20.7,0,0,...,0.16,0,0,2019-01-01 00:30:00,2019-01-01 00:30:00+00:00,2019-01-01 00:30:00-02:00,15,132.428602,192.435013,0.0
2,2,2019,1,1,1,0,0,20.6,0,0,...,0.16,0,0,2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2019-01-01 01:00:00-02:00,15,133.361499,183.099195,0.0
3,3,2019,1,1,1,30,0,20.5,0,0,...,0.16,0,0,2019-01-01 01:30:00,2019-01-01 01:30:00+00:00,2019-01-01 01:30:00-02:00,15,133.166114,173.623787,0.0
4,4,2019,1,1,2,0,0,20.4,0,0,...,0.16,0,0,2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2019-01-01 02:00:00-02:00,15,131.856204,164.429482,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,2019,12,31,21,30,0,21.4,0,0,...,0.16,0,0,2019-12-31 21:30:00,2019-12-31 21:30:00+00:00,2019-12-31 21:30:00-03:00,60,119.377718,222.640064,0.0
17516,17516,2019,12,31,22,0,0,21.3,0,0,...,0.16,0,0,2019-12-31 22:00:00,2019-12-31 22:00:00+00:00,2019-12-31 22:00:00-03:00,15,123.755981,216.380291,0.0
17517,17517,2019,12,31,22,30,0,21.2,0,0,...,0.16,0,0,2019-12-31 22:30:00,2019-12-31 22:30:00+00:00,2019-12-31 22:30:00-03:00,15,127.486511,209.238066,0.0
17518,17518,2019,12,31,23,0,0,21.1,0,0,...,0.16,0,0,2019-12-31 23:00:00,2019-12-31 23:00:00+00:00,2019-12-31 23:00:00-03:00,15,130.422052,201.206076,0.0


Tratamento dos dados: Criar Hour_sine e Hour_cosine e normalizar os outros dados

In [4]:
from sklearn.preprocessing import MinMaxScaler

def calcula_seno(hora,minuto):
    hour_sine = np.sin((2*np.pi*(hora*60+minuto))/1440)
    return hour_sine

def calcula_cosseno(hora,minuto):
    hour_cosine = np.cos((2*np.pi*(hora*60+minuto))/1440)
    return hour_cosine

senos = []
cossenos = []

for i, row in df_mes.iterrows(): # row representa cada linha do df
    hour_sine = calcula_seno(row['Hour'],row['Minute'])
    senos.append(hour_sine)
    hour_cosine = calcula_cosseno(row['Hour'],row['Minute'])
    cossenos.append(hour_cosine)

df_mes['Hour Sine'] = senos
df_mes['Hour Cosine'] = cossenos


cols = ['Unnamed: 0','Year','Day','Hour','Minute','Temperature','Cloud Type','Fill Flag','Surface Albedo','datetime_utc','datetime_local','zenith']
df_mes = df_mes.drop(cols,axis=1)
display(df_mes)

,Month,DHI,DNI,GHI,Solar Zenith Angle,Wind Speed,datetime,Angulo,azimuth,tracker_theta,Hour Sine,Hour Cosine
0,1,0,0,0,133.35,2.4,2019-01-01 00:00:00,15,201.247715,0.0,0.000000,1.000000
1,1,0,0,0,133.16,2.3,2019-01-01 00:30:00,15,192.435013,0.0,0.130526,0.991445
2,1,0,0,0,131.85,2.2,2019-01-01 01:00:00,15,183.099195,0.0,0.258819,0.965926
3,1,0,0,0,129.51,2.0,2019-01-01 01:30:00,15,173.623787,0.0,0.382683,0.923880
4,1,0,0,0,126.28,1.9,2019-01-01 02:00:00,15,164.429482,0.0,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,12,0,0,0,119.37,2.8,2019-12-31 21:30:00,60,222.640064,0.0,-0.608761,0.793353
17516,12,0,0,0,123.75,2.8,2019-12-31 22:00:00,15,216.380291,0.0,-0.500000,0.866025
17517,12,0,0,0,127.48,2.7,2019-12-31 22:30:00,15,209.238066,0.0,-0.382683,0.923880
17518,12,0,0,0,130.42,2.7,2019-12-31 23:00:00,15,201.206076,0.0,-0.258819,0.965926


In [5]:
df_mes.to_csv('df_mes.csv', index=False)